# <span style="color:blue"> UNet testing saving .png files

**Content under Creative Commons Attribution license CC-BY-NC-SA 4.0**  
**Code under GNU-GPL v3 License**  
**© 2023 Francesco Chiumento**

- Through this notebook, it is possible to utilize a previously saved checkpoint of the UNet neural network to segment femoral cartilages in 2D images extracted from MRI scans of a set of patients. In this version of the code, it is possible to provide the ground truth of the respective binary masks of the patients to visualize the qualitative results of the DICE indices of the obtained 2D segmentations.

- The ultimate goal is to obtain reconstructed segmentations in three-dimensional volumes from the 2D segmentations.

In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Library Import and Module Definition

In [ ]:
from slice_extraction import process_directory, clean_folder
from unet_segmentation import run_segmentation
from compositionMha import combine_slices_to_mha

import torch
import os

## Path and Settings Definition

In [ ]:
# Define paths and settings

images_dir = r"your/path/here"
masks_dir = r"your/path/here"
images_slices = r"your/path/here"
masks_slices = r"your/path/here"
checkpoint_path = r"your/path/here"
predictions = r"your/path/here"
segmentations = r"your/path/here"

spacing = (0.4121, 0.4121, 0.4)  # Update according to your image spacing

should_flip = True # Option to flip the images if needed

## Patient Directory Listing

In [ ]:
patient_dirs = os.listdir(images_dir)

## Device Selection

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Patient Data Processing Loop

In [ ]:
for patient in patient_dirs:
    image_mha_path = os.path.join(images_dir, patient)
    mask_mha_path = os.path.join(masks_dir, patient)
    
    # Clean the folders for slices and predictions before each patient
    clean_folder(images_slices)
    clean_folder(masks_slices)
    clean_folder(predictions)

    # Call the updated function with the specific file paths
    process_directory(image_mha_path, mask_mha_path, images_slices, masks_slices)

    run_segmentation(checkpoint_path, images_slices, masks_slices, predictions, device)

    combine_slices_to_mha(predictions, os.path.join(segmentations, f"{patient}_segmentation.mha"), spacing, should_flip)
   
    print(f"The process has been completed for the patient {patient}.")


## Computer system details

In [4]:
%reload_ext watermark
%watermark

Last updated: 2024-03-26T20:07:59.804054+01:00

Python implementation: CPython
Python version       : 3.11.4
IPython version      : 8.15.0

Compiler    : MSC v.1916 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 126 Stepping 5, GenuineIntel
CPU cores   : 8
Architecture: 64bit

